<a href="https://colab.research.google.com/github/olgasherbiena/KPI/blob/main/%D0%90%D0%9A_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!pip install kagglehub


In [50]:
import pandas as pd
import kagglehub
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

csv_path  = kagglehub.dataset_download("shantanudhakadd/email-spam-detection-dataset-classification")
csv_file = os.path.join(csv_path, 'spam.csv')
df = pd.read_csv(csv_file, encoding='latin1')

Using Colab cache for faster access to the 'email-spam-detection-dataset-classification' dataset.


Підготовка даних

In [51]:
df = df[['v1', 'v2']] # v1: 'label', v2: 'text'
df.columns = ['label', 'message']
df = df.iloc[:,:2]
df.columns = ['label', 'message']
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

# Векторизація текстових даних за допомогою TF-IDF
# TF-IDF перетворює текст на числові вектори, необхідні для LSH та Logistic Regression
tfidf = TfidfVectorizer(stop_words='english', max_features=2000) # Обмежуємо розмірність для прикладу
X_tfidf = tfidf.fit_transform(df['message']).toarray()
y = df['label_encoded'].values

Реалізація

In [52]:
import numpy as np

class LSH:
    def __init__(self, num_buckets, num_hash_functions, input_dim):
        self.num_buckets = num_buckets
        self.num_hash_functions = num_hash_functions
        self.input_dim = input_dim
        self.hash_functions = [self._generate_hash_function() for _ in range(num_hash_functions)]
        self.buckets = {i: [] for i in range(num_buckets)}

    def _generate_hash_function(self):
        return np.random.randn(self.input_dim)

    def hash(self, vector):
        hashes = []
        for hash_function in self.hash_functions:
            hash_value = np.dot(vector, hash_function)
            hash_value = hash_value / np.linalg.norm(hash_function)
            # Примітка: для LSH (на основі SimHash) часто використовують np.sign,
            # але ми залишаємо вашу логіку з np.astype(int) для цілих хешів.
            hash_value = hash_value.astype(int)
            hashes.append(hash_value)
        return hashes

    def add_vector(self, vector, data_index):
        hashes = self.hash(vector)
        for hash_value in hashes:
            self.buckets[hash_value % self.num_buckets].append((data_index, vector))

    def query(self, query_vector):
        query_hashes = self.hash(query_vector)
        similar_indices = set()

        for hash_value in query_hashes:
            bucket = self.buckets[hash_value % self.num_buckets]

            for index, vector in bucket:

                if np.array_equal(vector, query_vector):
                    continue
                similar_indices.add(index)
        return list(similar_indices)

# --- 2. ІНІЦІАЛІЗАЦІЯ ТА ВИКОНАННЯ ---

input_dim = X_tfidf.shape[1]
lsh_model = LSH(num_buckets=100, num_hash_functions=15, input_dim=input_dim)
for data_index, vector in enumerate(X_tfidf):
    lsh_model.add_vector(vector, data_index)

query_vector_lsh = X_tfidf[0]
similar_items_indices = lsh_model.query(query_vector_lsh)

print(f"Кількість хеш-функцій: 15")
print(f"Загальна кількість знайдених 'схожих' (унікальних) індексів: {len(similar_items_indices)}")

## ✅ Результат LSH після виправлення TypeError
Кількість хеш-функцій: 15
Загальна кількість знайдених 'схожих' (унікальних) індексів: 5571


In [53]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
test_queries = [
    "You can win a free car!! Click on the link"
]
X_test_queries = tfidf.transform(test_queries).toarray()
query_vector = X_test_queries[0] # Вектор Q1
similar_indices = lsh_model.query(query_vector)

if not similar_indices:
    print("LSH не знайшов жодних схожих індексів у своїх бакетах.")
else:
    # 1. Отримання векторів, які LSH вважає схожими
    X_similar = X_tfidf[similar_indices]

    # 2. Обчислення точної косинусної схожості
    similarities = cosine_similarity(query_vector.reshape(1, -1), X_similar).flatten()

    # 3. Створення DataFrame для сортування
    results = pd.DataFrame({
        'original_index': similar_indices,
        'similarity': similarities
    })

    # 4. Вибір топ-3 найсхожіших
    top_results = results.sort_values(by='similarity', ascending=False).head(3)

    print(f"\n## ✅ LSH: Топ-3 схожих повідомлень для '{test_queries[0][:30]}...'")

    for i, row in top_results.iterrows():
        original_idx = int(row['original_index'])
        similarity_score = row['similarity']

        # Виведення результатів з оригінального датасету
        label = df.iloc[original_idx]['label']
        message = df.iloc[original_idx]['message']

        print(f"\n{i+1}. Схожість: {similarity_score:.4f} (Індекс: {original_idx}, Клас: {label})")
        print(f"Текст: '{message}'")


## ✅ LSH: Топ-3 схожих повідомлень для 'You can win a free car!! Click...'

16. Схожість: 0.4614 (Індекс: 15, Клас: spam)
Текст: 'XXXMobileMovieClub: To use your credit, click the WAP link in the next txt message or click here>> http://wap. xxxmobilemovieclub.com?n=QJKGIGHJJGCBL'

5254. Схожість: 0.3312 (Індекс: 5253, Клас: ham)
Текст: 'Please tell me not all of my car keys are in your purse'

2805. Схожість: 0.3302 (Індекс: 2804, Клас: ham)
Текст: 'I think it's all still in my car'


Логістична регресія

In [54]:
# Розділення даних на тренувальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Ініціалізація та навчання моделі логістичної регресії
log_reg = LogisticRegression(solver='liblinear', random_state=42)
log_reg.fit(X_train, y_train)

# Прогнозування на тестовому наборі
y_pred = log_reg.predict(X_test)

# Оцінка результатів
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, target_names=le.classes_)

print(f"\n## 📈 Результати класифікації (Логістична регресія)")
print(f"Точність (Accuracy): {accuracy:.4f}")
print("\nМатриця плутанини (Confusion Matrix):")
print(conf_matrix)
print("\nЗвіт класифікації (Classification Report):")
print(class_report)


## 📈 Результати класифікації (Логістична регресія)
Точність (Accuracy): 0.9596

Матриця плутанини (Confusion Matrix):
[[962   3]
 [ 42 108]]

Звіт класифікації (Classification Report):
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98       965
        spam       0.97      0.72      0.83       150

    accuracy                           0.96      1115
   macro avg       0.97      0.86      0.90      1115
weighted avg       0.96      0.96      0.96      1115



In [55]:
# Використання навченої моделі log_reg
try:
    prediction = log_reg.predict(X_test_queries)[0]
    probability = log_reg.predict_proba(X_test_queries)[0, 1]
    predicted_label = 'SPAM' if prediction == 1 else 'HAM'

    print(f"\n## 🎯 Класифікація (Логістична регресія) для запиту Q4")
    print(f"Текст: '{test_queries[0]}'")
    print(f"Прогноз: **{predicted_label}** (Ймовірність SPAM: {probability:.4f})")
except NameError:
    print("Помилка: Змінна 'log_reg' (навчена модель) не визначена.")


## 🎯 Класифікація (Логістична регресія) для запиту Q4
Текст: 'You can win a free car!! Click on the link'
Прогноз: **HAM** (Ймовірність SPAM: 0.4524)
